In [ ]:
import pandas as pd
import glob
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
warnings.filterwarnings("ignore", category=FutureWarning)


files = glob.glob('./server_results/finetuning/*.csv')

dataset = 'OpenData'
files = glob.glob('./server_results/finetuning_valentine/'+dataset+'/*.csv')


In [58]:
def handle_matcher_names(df):




    substitutions = {
        'header_values_default': 'Default (Type)',
        'header_values_verbose': 'Verbose (Type)',
        'header_values_repeat': 'Repeat (Type)',
        'header_values_prefix': 'Prefix (Type)',
        'header_only': 'Header only',
        'header_values_verbose_notype': 'Verbose',
        'header_values_columnvaluepair_notype': 'Header-Value repeat',
        'header_header_values_repeat_notype': 'Repeat',
        'header_values_default_notype': 'Default',
    }

    df['encoding_mode'] = df['encoding_mode'].replace(substitutions)

    # print(df['encoding_mode'].unique())

    to_show = [
        #'Default (Type)',
        'Default',
        'Verbose',
        'Repeat',
        'Header-Value repeat',
        'Header only',
    ]

#     ['Default (Type)' 'Prefix (Type)' 'Repeat (Type)' 'Verbose (Type)'
#  'Column only' 'Verbose' 'ColVal Pairs' 'Repeat' 'Default']

    df = df[df['encoding_mode'].isin(to_show)].copy()


    
    
    df['sampling_mode'] = df['sampling_mode'].str.title()
    df['sampling_mode'] = df['sampling_mode'].str.replace('_Sampling', '')
    df = df[df['sampling_mode'] != 'Mixed'].copy()

    sampling_size = 10
    df = df[df['sampling_size'] == sampling_size].copy()

    df.rename(columns={'All_RecallAtSizeofGroundTruth': 'Recall@GT'}, inplace=True)
    df.rename(columns={'mrr': 'MRR'}, inplace=True)

    return df


def best_aggregate(df, metric='mrr'):

    df_grouped = df.groupby(['type'])[metric].agg(['mean', 'median', 'min', 'max', 'std']).reset_index()
    df_grouped = df_grouped.sort_values(by='mean', ascending=False)
    # print(df_grouped.head(20).to_string())
    return df_grouped



def plot(df, ax, x, y,  legend=False, save=None):
    

    order = df.groupby(x)['MRR'].mean().sort_values(ascending=False).index

    

    sns.barplot(
        data=df,
        x=x,
        y=y,
        ax=ax,
        palette=sns.color_palette("pastel"),  # Use a better palette
        order=order,
        # ci=None  # Disable confidence intervals
    )

    # sns.boxplot(
    #     data=df,
    #     x=x,
    #     hue=hue,
    #     y=y,
    #     ax=ax,
    #     palette=sns.color_palette("Set2"),  # Use a better palette
    #     order=order,
    #     hue_order=hue_order
    #     # , showfliers=False
    #     ,fliersize=0.5

    # )

    #ax.set_xlabel(x.replace('_', ' ').title(), fontsize=14)
    ax.set_xlabel('Column representation', fontsize=14)

    # ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_ylabel(y.upper(), fontsize=14)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)

    if legend:
        ax.set_xlabel('')

    if legend:
        ax.legend(
            loc='lower center',
            bbox_to_anchor=(0.48, 0.98),
            frameon=False
        )
    else:
        ax.get_legend().remove()

    sns.despine()



def plot_grouped_bar(df, x, metrics, figsize=(9, 8), save=None):

    fig, axis = plt.subplots(len(metrics),1 , figsize=figsize, sharex=True)

    for i, metric in enumerate(metrics):
        ax = axis[i]
        plot(df, ax, x,  metric, legend=(i==0), save=None)


    if save is not None:
        fig.savefig(save, format='pdf', dpi=300)

    plt.tight_layout()
    plt.show()
   

In [59]:
all_dfs_list = []
datasetnames = []



for file in files:
    datasetname = file.split('header_values_')[1].split('_results.csv')[0]
    datasetname= datasetname.replace('_results_.csv', '')

    print(datasetname)

    df = pd.read_csv(file)

    df['type'] = datasetname
    # df = handle_matcher_names(df)
    
    all_dfs_list.append(df)


all_dfs = pd.concat(all_dfs_list)

all_dfs = all_dfs.sort_values(by='type')

all_dfs.rename(columns={'All_RecallAtSizeofGroundTruth': 'Recall@GT'}, inplace=True)
all_dfs.rename(columns={'mrr': 'MRR'}, inplace=True)


df_ft = all_dfs[all_dfs['method'] == 'MatchMakerFT']
df_ft = df_ft.sort_values(by='type')

df_ftgpt = all_dfs[all_dfs['method'] == 'MatchMakerFTGPT']
df_ftgpt = df_ftgpt.sort_values(by='type')


repeat-exact
repeat-semantic
default-exact
verbose-exact_semantic
default-semantic
default-exact_semantic
verbose-semantic
verbose-exact


In [60]:
res = best_aggregate(df_ft, metric='MRR')
res.to_csv('best_aggregate_mrr.csv', index=False)
res

,type,mean,median,min,max,std
7,verbose-semantic,0.931890,1.000000,0.500000,1.0,0.102760
6,verbose-exact_semantic,0.930961,1.000000,0.500000,1.0,0.100579
5,verbose-exact,0.926812,1.000000,0.500000,1.0,0.103070
1,default-exact_semantic,0.904541,1.000000,0.100000,1.0,0.175365
2,default-semantic,0.900302,0.961648,0.166667,1.0,0.155215
0,default-exact,0.880245,0.938447,0.142857,1.0,0.181795
4,repeat-semantic,0.862165,0.916667,0.250000,1.0,0.158311
3,repeat-exact,0.833402,0.840744,0.250000,1.0,0.170518


In [61]:
res = best_aggregate(df_ft, metric='Recall@GT')
res.to_csv('best_aggregate_recall.csv', index=False)
res

,type,mean,median,min,max,std
1,default-exact_semantic,0.833069,0.909091,0.0,1.0,0.225698
4,repeat-semantic,0.795337,0.822134,0.0,1.0,0.222226
7,verbose-semantic,0.785420,0.833333,0.0,1.0,0.263219
2,default-semantic,0.781225,0.833333,0.0,1.0,0.261929
0,default-exact,0.775585,0.818182,0.0,1.0,0.261586
6,verbose-exact_semantic,0.774122,0.833333,0.0,1.0,0.273129
5,verbose-exact,0.763651,0.833333,0.0,1.0,0.281594
3,repeat-exact,0.746383,0.739130,0.0,1.0,0.260103


In [62]:
res  = best_aggregate(df_ftgpt, metric='MRR')
res.to_csv('best_aggregate_mrr_ftgpt.csv', index=False)
res

,type,mean,median,min,max,std
1,default-exact_semantic,0.964722,1.000000,0.666667,1.0,0.059797
7,verbose-semantic,0.959503,1.000000,0.333333,1.0,0.088920
5,verbose-exact,0.954525,1.000000,0.333333,1.0,0.096855
2,default-semantic,0.951454,1.000000,0.500000,1.0,0.076185
6,verbose-exact_semantic,0.948681,1.000000,0.250000,1.0,0.108834
0,default-exact,0.941798,1.000000,0.500000,1.0,0.082132
4,repeat-semantic,0.894750,1.000000,0.500000,1.0,0.128146
3,repeat-exact,0.878646,0.916667,0.500000,1.0,0.135757


In [63]:
res = best_aggregate(df_ftgpt, metric='Recall@GT')
res.to_csv('best_aggregate_recall_ftgpt.csv', index=False)
res

,type,mean,median,min,max,std
1,default-exact_semantic,0.846031,0.909091,0.0,1.0,0.225479
7,verbose-semantic,0.843810,0.909091,0.0,1.0,0.227399
2,default-semantic,0.840946,0.909091,0.0,1.0,0.227398
5,verbose-exact,0.839495,0.909091,0.0,1.0,0.228376
6,verbose-exact_semantic,0.834571,0.909091,0.0,1.0,0.227765
0,default-exact,0.831132,0.892045,0.0,1.0,0.227934
4,repeat-semantic,0.807825,0.833333,0.0,1.0,0.240095
3,repeat-exact,0.785028,0.818182,0.0,1.0,0.245766


In [64]:
metric1 = 'MRR'
metric2 = 'Recall@GT'
metrics = [metric1, metric2]

# plot_grouped_bar(all_dfs, x='type', metrics=metrics, figsize=(9, 8), save=None)